# Imports

In [ ]:
from snowflake.snowpark import Session
from snowflake.ml.registry import Registry
from snowflake.snowpark.context import get_active_session
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)

session = get_active_session()

# Connect to Registry and Feature Store

In [ ]:

reg = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema(),
)


fs = FeatureStore(
    session=session, 
    database=session.get_current_database(), 
    name=session.get_current_schema(), 
    default_warehouse=session.get_current_warehouse(),
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

print("Current Database: " + session.get_current_database())
print("Current Schema: " + session.get_current_schema())

In [ ]:
# fs.list_feature_views()
version = '1'
fv_instance_register_dates=fs.get_feature_view("real_estate_dates_fv", version)
fv_instance_register=fs.get_feature_view("real_estate_fv", version)

In [ ]:
reg.show_models()

In [ ]:
model = reg.get_model('Real_estate').version('RF_1st')

In [ ]:
model = reg.get_model('Real_estate').default

In [ ]:
predict_df = fs.generate_training_set(
    spine_df=session.sql(f"select ID from NFUM2981UKDEV01DB001.EDS.REAL_ESTATE_DATES where ID > 17337"),
    #"select ID from NFUM2981UKDEV01DB001.EDS.REAL_ESTATE_DATES where day == today()
    features=[fv_instance_register_dates,fv_instance_register]
).to_pandas()

In [ ]:
predict_df

# Apply Custom Transformations (optional, if applicable)

In [ ]:
import numpy as np
import pyproj
current_year = 2025
geodesic = pyproj.Geod(ellps='WGS84')
center = (52.22617782084895, 21.008312146369033)

def remove_outliers(df):
    df = df[(df.LONG < 21.3) & (df.LAT > 52.05)]
    df = df[df.BUILDING_FLOORS_NUM <=25]
    df = df[df.ROOMS_NUM <= 10]
    df['BUILDING_AGE'] = current_year - df['BUILD_YEAR']
    df['LOG_PRICE'] = np.log(df.PRICE)
    return df

# Forward Azimuth - the direction in which the apartment lies measured from a center
# Distance - distance from that center
# Together in a non-linear model these features reflect a zone/segment of the city in which the apartment is located

def adv_features(df):
    df['FWD_AZIMUTH_CENTER'] = df.apply(lambda x: 180 + geodesic.inv(x.LAT, x.LONG, center[0], center[1])[0],axis=1)
    return df

In [ ]:
# apply transformations
predict_df = remove_outliers(predict_df)
predict_df = adv_features(predict_df)

In [ ]:
final_variables =['SQM', 'BUILDING_AGE', 'BUILDING_FLOORS_NUM', 'ROOMS_NUM', 'FLOOR_NO', 'FWD_AZIMUTH_CENTER','WORD_BALCONY']

In [ ]:
pred_df = model.run(predict_df[final_variables], function_name='predict')

In [ ]:
pred_df

# Write results to a table

In [10]:
# TODO ...